In [87]:
# !pip install tqdm

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_rows',1000)
from datetime import datetime

# 1. 2017년도 id 리스트 뽑기
- id 모든 분기에 채우기
- 보증금 컬럼 추가
- 분양권/입주권 데이터로 결측치 채우기
- 6분기 이상 연속으로 거래가 없는 데이터 삭제
- 이전분기로 채우기
- id 리스트 저장

In [2]:
# 데이터 불러오기
df_mae = pd.read_csv('data/MasterMotherTable/df_mae_1721_11월04일17시34분.csv')
df_jeonwol = pd.read_csv('data/MasterMotherTable/df_jeonwol_1721_11월04일17시34분.csv')
# df_jeonwol['전세보증금'] = df_jeonwol['전세보증금'].astype(float)

df_mae.rename(columns={'quater':'quarter'}, inplace=True)
df_jeonwol.rename(columns={'quater':'quarter'}, inplace=True)

C:\Users\medici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### ※ `전월세구분`이 '월세'인 데이터 제거

In [3]:
# 전월세구분이 '월세'인 데이터 제거
df_jeonwol = df_jeonwol.drop(df_jeonwol[df_jeonwol.전월세구분 == '월세'].index.tolist())

# 리인덱싱
df_jeonwol.reset_index(drop=True, inplace=True)
df_jeonwol

,시군구,번지,본번,부번,단지명,전월세구분,전용면적_미제,계약년월,계약일,보증금,월세,층,건축년도,지역,전용면적_평,지번주소,new_Date,quarter,자치구_동,id
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201703,7,520.0,0,5,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2017-03-07,2017_1Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201707,15,510.0,0,3,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2017-07-15,2017_3Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201709,3,500.0,0,9,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2017-09-03,2017_3Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201709,20,480.0,0,7,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2017-09-20,2017_3Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,전세,80,201701,9,380.0,0,5,1987,서울,24.0,서울특별시 강남구 개포동 658-1,2017-01-09,2017_1Q,개포동,서울특별시 강남구 개포동_개포6차우성아파트1동~8동_80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208249,인천광역시 중구 항동7가,27-107,27,107,라이프비취3차,전세,85,202106,12,150.0,0,13,1990,인천,26.0,인천광역시 중구 항동7가 27-107,2021-06-12,2021_2Q,항동7가,인천광역시 중구 항동7가_라이프비취3차_85
1208250,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202101,7,20.0,0,3,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-01-07,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
1208251,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202102,21,25.0,0,3,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-02-21,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
1208252,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202103,16,16.0,0,2,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-03-16,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54


In [4]:
# 분양 데이터 불러오기
df_bunyang = pd.read_csv('data/분양권/비인도데프_분양데이터.csv')
df_bunyang

,id,quarter,전용면적_미제,거래금액,전용면적_평
0,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2016_1Q,NaN,NaN,NaN
1,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2016_2Q,NaN,NaN,NaN
2,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2016_3Q,NaN,NaN,NaN
3,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2016_4Q,NaN,NaN,NaN
4,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2017_1Q,NaN,NaN,NaN
...,...,...,...,...,...
64155,인천광역시 중구 중산동_하늘도시우미린2단지_85,2018_4Q,85.0,285.0,26.0
64156,인천광역시 중구 중산동_하늘도시우미린2단지_85,2019_1Q,NaN,NaN,NaN
64157,인천광역시 중구 중산동_하늘도시우미린2단지_85,2019_2Q,NaN,NaN,NaN
64158,인천광역시 중구 중산동_하늘도시우미린2단지_85,2019_3Q,85.0,313.5,26.0


In [5]:
# Grouping
df_mae_g = df_mae.groupby(['id','quarter'],as_index=False).mean()
# df_mae_g_2 = df_mae.groupby(['id','quarter','번지'],as_index=False).mean()

df_jeonwol_g = df_jeonwol.groupby(['id','quarter'],as_index=False).mean()
# df_jeonwol_g_2 = df_jeonwol.groupby(['id','quarter','번지'],as_index=False).mean()

## 1-1. `id`에 모든 분기 채우기

In [6]:
# 중복을 제거한 id 테이블
df_temp = df_mae_g.drop_duplicates(['id'],keep='first')

df_list = []

# 모든 id에 각각 2017_1Q~2021_2Q를 넣은 테이블을 생성하는 함수
def make_quarter(df):
    quarter_list = ['2017_1Q', '2017_2Q', '2017_3Q', '2017_4Q',
                   '2018_1Q', '2018_2Q', '2018_3Q', '2018_4Q',
                   '2019_1Q', '2019_2Q', '2019_3Q', '2019_4Q', 
                   '2020_1Q', '2020_2Q', '2020_3Q', '2020_4Q',
                   '2021_1Q', '2021_2Q']
    for item in tqdm(quarter_list):
        globals()['df_{}'.format(item)] = df.copy()
        globals()['df_{}'.format(item)]['quarter'] = str(item)
        df_list.append(globals()['df_{}'.format(item)])
        
    return

# df_temp에 함수 적용
make_quarter(df_temp)

# df_2017_1Q ~ df_2021_2Q 테이블 이어붙인 df_temp2 테이블 생성
df_temp2 = pd.concat(df_list,axis=0)
# df_temp2[['보증금','거래금액','주택담보대출금리','월세','전월세전환율','보증금']] = 0

# 여기서 id, quater만 뽑으면 id에 맞게 2017_1Q ~ 2021_2Q가 들어있는 데이터 완성.
df_temp2 = df_temp2[['id','quarter']]
df_temp2

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 462.77it/s]


,id,quarter
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q
13,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q
31,경기도 고양덕양구 고양동_삼성_60,2017_1Q
45,경기도 고양덕양구 고양동_삼성_85,2017_1Q
58,경기도 고양덕양구 고양동_윤창_59,2017_1Q
...,...,...
319310,인천광역시 중구 항동7가_라이프비취3차_164,2021_2Q
319320,인천광역시 중구 항동7가_라이프비취3차_85,2021_2Q
319338,인천광역시 중구 항동7가_연안아파트_54,2021_2Q
319355,인천광역시 중구 항동7가_연안아파트_55,2021_2Q


In [7]:
# df_temp2 테이블에 df_mae_g를 right join
df_merge = pd.merge(df_mae_g,df_temp2[['id','quarter']], how='right',on=['id','quarter'])

# id, quater 기준으로 재배열
df_merge = df_merge.sort_values(['id','quarter'])
df_merge.reset_index(drop=True, inplace=True)

df_merge.drop(['계약년월','계약일','층'], axis=1, inplace=True)
df_merge

,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,NaN,NaN,NaN,NaN,NaN,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,454.0,0.0,60.0,203.000000,2007.0,18.0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,454.0,0.0,60.0,200.666667,2007.0,18.0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,454.0,0.0,60.0,209.000000,2007.0,18.0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,454.0,0.0,60.0,209.000000,2007.0,18.0
...,...,...,...,...,...,...,...,...
767605,인천광역시 중구 항동7가_연안아파트_81,2020_2Q,NaN,NaN,NaN,NaN,NaN,NaN
767606,인천광역시 중구 항동7가_연안아파트_81,2020_3Q,NaN,NaN,NaN,NaN,NaN,NaN
767607,인천광역시 중구 항동7가_연안아파트_81,2020_4Q,91.0,2.0,81.0,200.000000,1983.0,24.0
767608,인천광역시 중구 항동7가_연안아파트_81,2021_1Q,91.0,2.0,81.0,208.000000,1983.0,24.0


## 1-2. `보증금` 컬럼 추가

In [8]:
# 매매데이터와 전월세데이터 merge : '보증금' 컬럼 추가
df_merge = pd.merge(df_merge,df_jeonwol_g[['id','quarter','보증금']] , how='left', on=['id','quarter'])

# id, quater 기준으로 재배열
df_merge = df_merge.sort_values(['id','quarter'])
df_merge.reset_index(drop=True, inplace=True)

df_merge

,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,NaN,NaN,NaN,NaN,NaN,NaN,166.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,454.0,0.0,60.0,203.000000,2007.0,18.0,NaN
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,454.0,0.0,60.0,200.666667,2007.0,18.0,175.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000
...,...,...,...,...,...,...,...,...,...
767605,인천광역시 중구 항동7가_연안아파트_81,2020_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767606,인천광역시 중구 항동7가_연안아파트_81,2020_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767607,인천광역시 중구 항동7가_연안아파트_81,2020_4Q,91.0,2.0,81.0,200.000000,1983.0,24.0,NaN
767608,인천광역시 중구 항동7가_연안아파트_81,2021_1Q,91.0,2.0,81.0,208.000000,1983.0,24.0,NaN


## 1-3. 분양권/입주권 데이터로 결측치 채우기

In [9]:
# 분양이 영어로 sub division이라고 합니다.
df_subDivision = pd.read_csv('data/분양권/비인도데프_분양데이터.csv')

# 전용면적_미제, 전용면적_평 컬럼 삭제
df_subDivision.drop(['전용면적_미제','전용면적_평'], axis=1, inplace=True)

# 분양권/입주권 가격이 없는 데이터 삭제. (113234-38454 = 74780)
df_subDivision.dropna(inplace=True)
df_subDivision.reset_index(drop=True, inplace=True)

# 컬럼명: 거래금액 -> 분양권/입주권
df_subDivision.rename(columns={'거래금액':'분양권/입주권'}, inplace=True)

# 컬럼명: quater -> quarter
df_subDivision.rename(columns={'quater':'quarter'}, inplace=True)

df_subDivision

,id,quarter,분양권/입주권
0,경기도 고양덕양구 덕은동_고양덕은지구 A5블록 대방노블랜드_85,2019_3Q,709.050000
1,경기도 고양덕양구 도내동_고양원흥동일스위트_55,2017_2Q,391.250000
2,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2016_3Q,380.736802
3,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2016_4Q,382.124182
4,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_1Q,383.206250
...,...,...,...
16354,인천광역시 중구 중산동_하늘도시 한라비발디_102,2017_3Q,355.000000
16355,인천광역시 중구 중산동_하늘도시 한라비발디_102,2019_2Q,325.000000
16356,인천광역시 중구 중산동_하늘도시우미린1단지_49,2017_3Q,200.000000
16357,인천광역시 중구 중산동_하늘도시우미린2단지_85,2018_4Q,285.000000


In [10]:
# 매메 데이터와 id, quater가 일치하는 데이터에 분양권/입주권 join
df_merge = pd.merge(df_merge, df_subDivision[['id','quarter','분양권/입주권']], how ='left', on= ['id','quarter'])
df_merge

,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,분양권/입주권
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,NaN,NaN,NaN,NaN,NaN,NaN,166.666667,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,454.0,0.0,60.0,203.000000,2007.0,18.0,NaN,NaN
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,454.0,0.0,60.0,200.666667,2007.0,18.0,175.000000,NaN
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000,NaN
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
767605,인천광역시 중구 항동7가_연안아파트_81,2020_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767606,인천광역시 중구 항동7가_연안아파트_81,2020_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767607,인천광역시 중구 항동7가_연안아파트_81,2020_4Q,91.0,2.0,81.0,200.000000,1983.0,24.0,NaN,NaN
767608,인천광역시 중구 항동7가_연안아파트_81,2021_1Q,91.0,2.0,81.0,208.000000,1983.0,24.0,NaN,NaN


In [11]:
# 거래금액이 없고 분양권/입주권이 존재하는 데이터 인덱스
idx_bunyang = df_merge[(df_merge['분양권/입주권'].isna() == False) & (df_merge.거래금액.isna() == True)].index
print(idx_bunyang.shape)
# 거래금액이 없고 분양권/입주권이 존재하는 데이터를 따로 저장해놓는다. 초기 분양권/입주권으로 결측치를 채울 때 쓸 예정
insert_bunyang = df_merge[(df_merge['분양권/입주권'].isna() == False) & (df_merge.거래금액.isna() == True)]

# 거래금액이 없고 분양권/입주권이 존재하는 데이터에 대해 분양권/입주권을 거래금액으로 저장
df_merge.loc[idx_bunyang, '거래금액'] = df_merge.loc[idx_bunyang, '분양권/입주권']

# 분양권/입주권 삭제. 다시 df_merge로 정의
df_merge.drop('분양권/입주권', axis=1, inplace=True)
display(df_merge)

# df_merge에 잘 적용되었는지 확인
display(df_merge.loc[idx_bunyang])

(7253,)


,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,NaN,NaN,NaN,NaN,NaN,NaN,166.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,454.0,0.0,60.0,203.000000,2007.0,18.0,NaN
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,454.0,0.0,60.0,200.666667,2007.0,18.0,175.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,454.0,0.0,60.0,209.000000,2007.0,18.0,170.000000
...,...,...,...,...,...,...,...,...,...
767605,인천광역시 중구 항동7가_연안아파트_81,2020_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767606,인천광역시 중구 항동7가_연안아파트_81,2020_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767607,인천광역시 중구 항동7가_연안아파트_81,2020_4Q,91.0,2.0,81.0,200.000000,1983.0,24.0,NaN
767608,인천광역시 중구 항동7가_연안아파트_81,2021_1Q,91.0,2.0,81.0,208.000000,1983.0,24.0,NaN


,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
1080,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_1Q,NaN,NaN,NaN,383.206250,NaN,NaN,NaN
1081,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_2Q,NaN,NaN,NaN,393.983056,NaN,NaN,NaN
1082,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_3Q,NaN,NaN,NaN,404.584103,NaN,NaN,NaN
1083,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_4Q,NaN,NaN,NaN,413.975100,NaN,NaN,NaN
1084,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2018_1Q,NaN,NaN,NaN,414.315833,NaN,NaN,305.516129
...,...,...,...,...,...,...,...,...,...
767075,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_2Q,NaN,NaN,NaN,329.000000,NaN,NaN,NaN
767076,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_3Q,NaN,NaN,NaN,329.915000,NaN,NaN,NaN
767077,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_4Q,NaN,NaN,NaN,325.500000,NaN,NaN,NaN
767078,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2019_1Q,NaN,NaN,NaN,323.185000,NaN,NaN,20.000000


## 1-4. 6분기 이상 거래가 없었던 `id` 삭제

In [12]:
# n에 따라 n+1분기 이상 거래가 없는 데이터 삭제 

# 비교열 생성 함수
def id_reduce_quater(n):
    for count in range(n):
        count += 1
        df_merge['비교열'+str(count)]= df_merge['거래금액'].shift(int(count))

        if count > 8:
            df_merge.loc[df_merge['quarter'] == '2019_'+str(count-8)+'Q','바뀜'] = 1
        elif count > 4:
            df_merge.loc[df_merge['quarter'] == '2018_'+str(count-4)+'Q','바뀜'] = 1
        else:
            df_merge.loc[df_merge['quarter'] == '2017_'+str(count)+'Q','바뀜'] = 1
        

# 6분기이상 거래가 없는 데이터 삭제
id_reduce_quater(5)

# '거래금액','비교열1','비교열2','비교열3','비교열4','비교열5','바뀜' 컬럼이 모두 NaN인 데이터를 제거하고 남은 데이터의 인덱스
list_of_true = df_merge[['거래금액','비교열1','비교열2','비교열3','비교열4','비교열5','바뀜']].dropna(thresh=1).index

# 남은 데이터의 아이디에 대해 18분기 미만의 id -> 삭제 대상
id_under_18 = df_merge.loc[list_of_true]['id'].value_counts() < 18
index_under_18 = id_under_18[id_under_18 == True].index

# 전체 아이디에서 18분기미만의 데이터 제외한 리스트
list_18q = list(set(df_merge['id'].unique())-set(index_under_18))

# 결과
df_merge = df_merge[df_merge['id'].isin(list_18q)]
df_merge.reset_index(drop=True, inplace=True)


# 본번, 부번, 건축년도, 전용면적_미제, 전용면적_평 결측치 채워넣기
df_merge[['본번','부번','건축년도','전용면적_미제','전용면적_평']] = df_merge.groupby('id')[['본번','부번','건축년도','전용면적_미제','전용면적_평']].ffill().bfill()

# 불필요 피처 삭제
df_merge = df_merge.drop(['본번','부번','비교열1','비교열2','비교열3','비교열4','비교열5','바뀜'],axis=1)
df_merge

C:\Users\medici\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,NaN
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000
...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333


In [16]:
df_merge.drop_duplicates()

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,NaN
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000
...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333


In [13]:
df_merge.isna().sum()

id              0
quarter         0
전용면적_미제         0
거래금액       114568
건축년도            0
전용면적_평          0
보증금        135489
dtype: int64

## 1-5.결측치 이전분기로 채워넣기

In [106]:
# 결측치 확인
df_merge.isna().sum()

id              0
quarter         0
전용면적_미제         0
거래금액       114568
건축년도            0
전용면적_평          0
보증금        135489
dtype: int64

In [107]:
# 이전분기 채우기
df_merge['거래금액'] = df_merge.groupby('id')['거래금액'].ffill()
df_merge[['보증금']] = df_merge.groupby('id')[['보증금']].ffill()


display(df_merge)

# 결측치 확인
display(df_merge.isna().sum())

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000
...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333


id             0
quarter        0
전용면적_미제        0
거래금액       12934
건축년도           0
전용면적_평         0
보증금        34340
dtype: int64

In [108]:
# 결측치 확인
df_merge.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액       12934
건축년도           0
전용면적_평         0
보증금        34340
dtype: int64

## ※ 매매상승률, 전세가 상승률 컬럼 추가

In [109]:
# 이전거래금액 컬럼 생성.
df_merge['이전거래금액'] = df_merge['거래금액'].shift()
# 2017년도 첫 분기는 0이 되게 설정
df_merge.loc[(df_merge.quarter == '2017_1Q'), '이전거래금액'] = None

# 이전보증금 컬럼 생성
df_merge['이전보증금'] = df_merge.보증금.shift()
# 2017년도 첫 분기는 0이 되게 설정
df_merge.loc[(df_merge.quarter == '2017_1Q'), '이전보증금'] = None


# 매매상승률, 전세상승률 컬럼 생성
df_merge['매매상승률'] = (df_merge.거래금액 - df_merge.이전거래금액) * 100 / df_merge.이전거래금액 
df_merge['전세상승률'] = (df_merge.보증금 - df_merge.이전보증금) * 100 / df_merge.이전보증금

# 2017년도 첫 분기는 0이 되게 설정
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.매매상승률.isna()), '매매상승률'] = 0
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.전세상승률.isna()), '전세상승률'] = 0

# 이전거래금액, 이전보증금 삭제
df_merge.drop(['이전거래금액','이전보증금'], axis=1, inplace=True)
df_merge.head(100)

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,NaN,0.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000
5,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_2Q,60.0,198.000000,2007.0,18.0,171.666667,-5.263158,0.980392
6,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_3Q,60.0,202.000000,2007.0,18.0,185.000000,2.020202,7.766990
7,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_4Q,60.0,202.000000,2007.0,18.0,185.000000,0.000000,0.000000
8,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2019_1Q,60.0,196.833333,2007.0,18.0,180.000000,-2.557756,-2.702703
9,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2019_2Q,60.0,196.833333,2007.0,18.0,180.000000,0.000000,0.000000


## 1-5. id 리스트 저장

In [110]:
# 2017년도 1분기 id 리스트 저장
id_list_2017 = df_merge[df_merge.quarter == '2017_1Q']['id'].unique()

# id 개수
print(len(id_list_2017))

# 2017~2021년 데이터 저장
df_merge_2017 = df_merge.copy()

# 2017년 1분기이면서 거래금액이 없는 id 리스트
nan_mae_17_1Q = df_merge_2017[(df_merge_2017.quarter=='2017_1Q') & (df_merge_2017.거래금액.isna())]['id'].unique()

# 2017년도 1분기이면서 보증금이 없는 id 리스트
nan_jeonwol_17_1Q = df_merge_2017[(df_merge_2017.quarter=='2017_1Q') & (df_merge_2017.보증금.isna())]['id'].unique()

21516


#### 2017~2021년도 데이터 결측치 확인

In [111]:
# 2017~2021년도 데이터 결측치 확인
display(df_merge_2017.isna().sum())

# 2017년도 1분기 데이터 결측치 확인
display(df_merge_2017.loc[df_merge_2017.quarter=='2017_1Q'].isna().sum())

id             0
quarter        0
전용면적_미제        0
거래금액       12934
건축년도           0
전용면적_평         0
보증금        34340
매매상승률      34450
전세상승률      55514
dtype: int64

id             0
quarter        0
전용면적_미제        0
거래금액        7059
건축년도           0
전용면적_평         0
보증금         8336
매매상승률      21516
전세상승률      21516
dtype: int64

# 2. 2014~2021 데이터로 2017년도 1분기 데이터 얻기
- id 모든 분기에 채우기
- 보증금 컬럼 추가
- 이전 분기로 채우기
- id_list_2017 에 id 리스트에 있는 id만 2017년도 1분기 데이터 저장하기

In [112]:
# 데이터 불러오기
df_mae = pd.read_csv('data/MasterMotherTable/df_mae_1421_11월04일17시29분.csv')
df_jeonwol = pd.read_csv('data/MasterMotherTable/df_jeonwol_1421_11월04일17시29분.csv')

# quater -> quarter
df_mae.rename(columns={'quater':'quarter'}, inplace=True)
df_jeonwol.rename(columns={'quater':'quarter'}, inplace=True)

C:\Users\medici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [113]:
df_mae.shape

(2075298, 18)

In [114]:
print(df_jeonwol.shape)
df_jeonwol = df_jeonwol.drop(df_jeonwol[df_jeonwol.전월세구분 == '월세'].index.tolist())
print(df_jeonwol.shape)
df_jeonwol.reset_index(drop=True, inplace=True)
df_jeonwol

(2849336, 20)
(1912090, 20)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적_미제,계약년월,계약일,보증금,월세,층,건축년도,지역,전용면적_평,지번주소,new_Date,quarter,자치구_동,id
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201401,21,330.0,0,2.0,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-01-21,2014_1Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201405,9,390.0,0,9.0,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-05-09,2014_2Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201406,24,350.0,0,9.0,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-06-24,2014_2Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201407,31,370.0,0,3.0,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-07-31,2014_3Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,78,201409,1,390.0,0,8.0,1988,서울,24.0,서울특별시 강남구 개포동 655-2,2014-09-01,2014_3Q,개포동,서울특별시 강남구 개포동_개포2차현대아파트(220)_78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912085,인천광역시 중구 항동7가,27-107,27,107,라이프비취3차,전세,85,202106,12,150.0,0,13.0,1990,인천,26.0,인천광역시 중구 항동7가 27-107,2021-06-12,2021_2Q,항동7가,인천광역시 중구 항동7가_라이프비취3차_85
1912086,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202101,7,20.0,0,3.0,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-01-07,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
1912087,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202102,21,25.0,0,3.0,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-02-21,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54
1912088,인천광역시 중구 항동7가,91-2,91,2,연안아파트,전세,54,202103,16,16.0,0,2.0,1983,인천,16.0,인천광역시 중구 항동7가 91-2,2021-03-16,2021_1Q,항동7가,인천광역시 중구 항동7가_연안아파트_54


In [115]:
# Grouping
df_mae_g = df_mae.groupby(['id','quarter'],as_index=False).mean()
df_jeonwol_g = df_jeonwol.groupby(['id','quarter'],as_index=False).mean()

## 2-1. id 모든 분기에 채우기

In [116]:
# 중복을 제거한 id 테이블
df_temp = df_mae_g.drop_duplicates(['id'],keep='first')

df_list = []

# 모든 id에 각각 2017_1Q~2021_2Q를 넣은 테이블을 생성하는 함수
def make_quarter(df):
    quarter_list = ['2014_1Q', '2014_2Q', '2014_3Q', '2014_4Q',
                   '2015_1Q', '2015_2Q', '2015_3Q', '2015_4Q',
                   '2016_1Q', '2016_2Q', '2016_3Q', '2016_4Q',
                   '2017_1Q', '2017_2Q', '2017_3Q', '2017_4Q',
                   '2018_1Q', '2018_2Q', '2018_3Q', '2018_4Q',
                   '2019_1Q', '2019_2Q', '2019_3Q', '2019_4Q', 
                   '2020_1Q', '2020_2Q', '2020_3Q', '2020_4Q',
                   '2021_1Q', '2021_2Q']
    for item in tqdm(quarter_list):
        globals()['df_{}'.format(item)] = df.copy()
        globals()['df_{}'.format(item)]['quarter'] = str(item)
        df_list.append(globals()['df_{}'.format(item)])
        
    return

# df_temp에 함수 적용
make_quarter(df_temp)

# df_2017_1Q ~ df_2021_2Q 테이블 이어붙인 df_temp2 테이블 생성
df_temp2 = pd.concat(df_list,axis=0)
# df_temp2[['보증금','거래금액','주택담보대출금리','월세','전월세전환율','보증금']] = 0

# 여기서 id, quater만 뽑으면 id에 맞게 2017_1Q ~ 2021_2Q가 들어있는 데이터 완성.
df_temp2 = df_temp2[['id','quarter']]
df_temp2

100%|██████████| 30/30 [00:00<00:00, 334.12it/s]


,id,quarter
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_1Q
25,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2014_1Q
55,경기도 고양덕양구 고양동_삼성_60,2014_1Q
80,경기도 고양덕양구 고양동_삼성_85,2014_1Q
101,경기도 고양덕양구 고양동_윤창_59,2014_1Q
...,...,...
537404,인천광역시 중구 항동7가_라이프비취3차_164,2021_2Q
537421,인천광역시 중구 항동7가_라이프비취3차_85,2021_2Q
537451,인천광역시 중구 항동7가_연안아파트_54,2021_2Q
537480,인천광역시 중구 항동7가_연안아파트_55,2021_2Q


In [117]:
# df_temp2 테이블에 df_mae_g를 right join
df_merge_2 = pd.merge(df_mae_g,df_temp2[['id','quarter']], how='right',on=['id','quarter'])

# id, quater 기준으로 재배열
df_merge_2 = df_merge_2.sort_values(['id','quarter'])
df_merge_2.reset_index(drop=True,inplace=True)

df_merge_2

,id,quarter,본번,부번,전용면적_미제,계약년월,계약일,거래금액,층,건축년도,전용면적_평
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_1Q,454.0,0.0,60.0,201402.500000,8.0,172.5,9.250000,2007.0,18.0
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_2Q,454.0,0.0,60.0,201405.666667,18.0,164.0,4.666667,2007.0,18.0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_3Q,454.0,0.0,60.0,201407.800000,23.0,165.4,5.400000,2007.0,18.0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_4Q,454.0,0.0,60.0,201411.000000,7.0,164.0,17.000000,2007.0,18.0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_1Q,454.0,0.0,60.0,201502.600000,12.6,174.2,5.200000,2007.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
1410775,인천광역시 중구 항동7가_연안아파트_81,2020_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410776,인천광역시 중구 항동7가_연안아파트_81,2020_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410777,인천광역시 중구 항동7가_연안아파트_81,2020_4Q,91.0,2.0,81.0,202010.000000,6.0,200.0,2.000000,1983.0,24.0
1410778,인천광역시 중구 항동7가_연안아파트_81,2021_1Q,91.0,2.0,81.0,202102.000000,4.0,208.0,2.000000,1983.0,24.0


## 2-2. `보증금` 컬럼 추가

In [118]:
# 매매데이터와 전월세데이터 merge : '보증금' 컬럼 추가
df_merge_2 = pd.merge(df_merge_2,df_jeonwol_g[['id','quarter','보증금']] , how='left', on=['id','quarter'])

# id, quater 기준으로 재배열
df_merge_2 = df_merge_2.sort_values(['id','quarter'])
df_merge_2.reset_index(drop=True, inplace=True)

# 2017~2021 데이터 id에 해당하는 데이터만 사용
df_merge_2 = df_merge_2[df_merge_2.id.isin(list_18q)]
df_merge_2.reset_index(drop=True, inplace=True)


df_merge_2

,id,quarter,본번,부번,전용면적_미제,계약년월,계약일,거래금액,층,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_1Q,454.0,0.0,60.0,201402.500000,8.000000,172.500000,9.250000,2007.0,18.0,121.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_2Q,454.0,0.0,60.0,201405.666667,18.000000,164.000000,4.666667,2007.0,18.0,120.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_3Q,454.0,0.0,60.0,201407.800000,23.000000,165.400000,5.400000,2007.0,18.0,NaN
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_4Q,454.0,0.0,60.0,201411.000000,7.000000,164.000000,17.000000,2007.0,18.0,NaN
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_1Q,454.0,0.0,60.0,201502.600000,12.600000,174.200000,5.200000,2007.0,18.0,130.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
645475,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,91.0,2.0,54.0,202005.277778,19.055556,229.277778,3.333333,1983.0,16.0,21.666667
645476,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,91.0,2.0,54.0,202007.666667,14.666667,210.000000,3.000000,1983.0,16.0,18.285714
645477,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,91.0,2.0,54.0,202010.923077,20.384615,204.692308,3.384615,1983.0,16.0,20.000000
645478,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,91.0,2.0,54.0,202101.960000,15.840000,232.360000,3.440000,1983.0,16.0,20.333333


## 2-3. 결측치 이전 분기로 채워넣기
- 6분기 이상 거래가 없는 데이터를 삭제하기 전에 이전 분기로 채워넣는다.
- 이때, 6분기 이상 거래가 없는 데이터도 이전 분기 데이터로 채워넣어질 수 있다.
- 따라서 지금부터 생기는 데이터는 2017_1Q 데이터만 유의미하다.
- 여기서 2017년도 1분기 데이터만 뽑아낸 다음 Mother Table을 다시 불러올 것이다.

In [119]:
# 결측치 확인
df_merge_2.isna().sum()

id              0
quarter         0
본번         202133
부번         202133
전용면적_미제    202133
계약년월       202133
계약일        202133
거래금액       202133
층          202133
건축년도       202133
전용면적_평     202133
보증금        235574
dtype: int64

In [120]:
# 본번, 부번, 건축년도, 전용면적_미제, 전용면적_평 결측치 채워넣기
df_merge_2[['본번','부번','건축년도','전용면적_미제','전용면적_평']] = df_merge_2.groupby('id')[['본번','부번','건축년도','전용면적_미제','전용면적_평']].ffill().bfill()

# 불필요 피처 삭제
df_merge_2 = df_merge_2.drop(['층','본번','부번','계약년월','계약일'],axis=1)


# 이전분기 채우기
df_merge_2['거래금액'] = df_merge_2.groupby('id')['거래금액'].ffill()
df_merge_2[['보증금']] = df_merge_2.groupby('id')[['보증금']].ffill()

df_merge_2

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_1Q,60.0,172.500000,2007.0,18.0,121.666667
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_2Q,60.0,164.000000,2007.0,18.0,120.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_3Q,60.0,165.400000,2007.0,18.0,120.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_4Q,60.0,164.000000,2007.0,18.0,120.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_1Q,60.0,174.200000,2007.0,18.0,130.000000
...,...,...,...,...,...,...,...
645475,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667
645476,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714
645477,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000
645478,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333


In [121]:
# 결측치 확인
df_merge_2.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액       47040
건축년도           0
전용면적_평         0
보증금        51937
dtype: int64

#### 2017~2021 데이터의 ID 리스트에 해당하는 ID만 검색하여 결측치 확인  

**거래금액 : 9634개**

**보증금 : 12339개**

In [122]:
# 2014~2021년 데이터를 df_merge_2014에 저장.
df_merge_2014 = df_merge_2.copy()

# 2017~2021년 데이터의 id와 일치하는 데이터만 결측치 확인하기
quarter_list = ['2017_1Q', '2017_2Q', '2017_3Q', '2017_4Q',
                   '2018_1Q', '2018_2Q', '2018_3Q', '2018_4Q',
                   '2019_1Q', '2019_2Q', '2019_3Q', '2019_4Q', 
                   '2020_1Q', '2020_2Q', '2020_3Q', '2020_4Q',
                   '2021_1Q', '2021_2Q']

# 2014~2021 데이터에서 2017~2021 id 리스트에 해당하는 데이터 결측치 확인
df_merge_2014.loc[(df_merge_2014.quarter.isin(quarter_list)) & df_merge_2014.id.isin(id_list_2017)].isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        9634
건축년도           0
전용면적_평         0
보증금        12339
dtype: int64

#### 2017_1Q  남은 결측치 확인하기.
**거래금액 : 839개**

**보증금 : 1701개**

In [123]:
# df_merge 안에서 2017년도 1분기 NaN List 에 해당하는 id만 선별
df_temp_mae = df_merge_2014[df_merge_2014.id.isin(nan_mae_17_1Q)]
df_temp_jeonwol = df_merge_2014[df_merge_2014.id.isin(nan_jeonwol_17_1Q)]

# 거기서 거래금액 0원인거
display(df_temp_mae[(df_temp_mae.quarter == '2017_1Q') & (df_temp_mae.거래금액.isna())].shape)

# 거기서 보증금 0원인거
display(df_temp_jeonwol[(df_temp_jeonwol.quarter == '2017_1Q') & (df_temp_jeonwol.보증금.isna())].shape)

(839, 7)

(1701, 7)

### 매매상승률, 전세상승률

In [124]:
# 이전거래금액 컬럼 생성.
df_merge_2014['이전거래금액'] = df_merge_2014['거래금액'].shift()
# 2017년도 첫 분기는 0이 되게 설정
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.거래금액.isna()), '이전거래금액'] = None

# 이전보증금 컬럼 생성
df_merge_2014['이전보증금'] = df_merge_2014.보증금.shift()
# 2017년도 첫 분기는 0이 되게 설정
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.보증금.isna()), '이전보증금'] = None


# 매매상승률, 전세상승률 컬럼 생성
df_merge_2014['매매상승률'] = (df_merge_2014.거래금액 - df_merge_2014.이전거래금액) * 100 / df_merge_2014.이전거래금액 
df_merge_2014['전세상승률'] = (df_merge_2014.보증금 - df_merge_2014.이전보증금) * 100 / df_merge_2014.이전보증금

# 2017년도 첫 분기는 0이 되게 설정
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.매매상승률.isna()), '매매상승률'] = 0
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.전세상승률.isna()), '전세상승률'] = 0

# 이전거래금액, 이전보증금 삭제
df_merge_2014.drop(['이전거래금액','이전보증금'], axis=1, inplace=True)
df_merge_2014.head(100)

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_1Q,60.0,172.500000,2007.0,18.0,121.666667,NaN,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_2Q,60.0,164.000000,2007.0,18.0,120.000000,-4.927536,-1.369863
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_3Q,60.0,165.400000,2007.0,18.0,120.000000,0.853659,0.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2014_4Q,60.0,164.000000,2007.0,18.0,120.000000,-0.846433,0.000000
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_1Q,60.0,174.200000,2007.0,18.0,130.000000,6.219512,8.333333
5,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_2Q,60.0,177.500000,2007.0,18.0,130.000000,1.894374,0.000000
6,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_3Q,60.0,178.750000,2007.0,18.0,140.000000,0.704225,7.692308
7,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2015_4Q,60.0,192.000000,2007.0,18.0,150.000000,7.412587,7.142857
8,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2016_1Q,60.0,190.000000,2007.0,18.0,150.000000,-1.041667,0.000000
9,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2016_2Q,60.0,187.000000,2007.0,18.0,160.000000,-1.578947,6.666667


# 3. 2017~2021 데이터에서 2017년 1분기 데이터 바꾸기
- 2017~2021 데이터에서 2017_1Q이면서 거래금액이 없는 데이터 -> maeNull_2017
- 2017~2021 데이터에서 2017_1Q이면서 보증금이 없는 데이터 -> jeonNull_2017
- 2014~2021 데이터에서 2017_1Q인 데이터 -> df_2014_17_1Q


1. `maeNull_2017`와 `df_2014_17_1Q`를 inner join -> `maeNull_2017`에서 `df_2014_17_1Q`와 `id`, `quarter`가 일치하는 거래금액의 결측치를 채움-> `df_inner_mae`
2. `df_merge_2017`의 2017년도 1분기에 결측치가 아닌 데이터와 `df_inner_mae`를 concat -> `df_result_mae`
3. `jeonNull_2017`와 `df_2014_17_1Q`를 inner join -> `jeonNull_2017`에서 `df_2014_17_1Q`와 `id`, `quarter`가 일치하는 보증금의 결측치를 채움-> `df_inner_jeon`
4. `df_merge_2017`의 2017년도 1분기에 결측치가 아닌 데이터와 `df_inner_jeon`를 concat -> `df_result_jeon`
5. `df_result_mae`의 거래금액과 `df_inner_jeon`의 보증금을 가진 테이블 생성 -> 수정된 2017_1Q 데이터 `df_result`
6. `df_result`와 `df_merge_2017` 2017_2Q ~ 2021_2Q 데이터를 병합. 바꾸기 성공!

In [125]:
# 2017~2021 데이터에서 2017_1Q이면서 거래금액이 없는 데이터 -> maeNull_2017
maeNull_2017 = df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.거래금액.isna())]

# 2017~2021 데이터에서 2017_1Q이면서 매매상승률이 없는 데이터 -> maeNull_2017
maeNull_2017_2 = df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.매매상승률.isna())]

# 2017~2021 데이터에서 2017_1Q이면서 보증금 없는 데이터 -> jeonNull_2017
jeonNull_2017 = df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.보증금.isna())]

# 2017~2021 데이터에서 2017_1Q이면서 전세상승률이 없는 데이터 -> maeNull_2017
jeonNull_2017_2 = df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.전세상승률.isna())]

# 2014~2021 데이터에서 2017_1Q인 데이터 -> df_2014_17_1Q
df_2014_17_1Q = df_merge_2014[df_merge_2014.quarter=='2017_1Q']

# maeNull_2017와 df_2014_17_1Q를 inner join
df_inner_mae = pd.merge(maeNull_2017, df_2014_17_1Q, how ='inner', on =['id','quarter'])
df_inner_mae

,id,quarter,전용면적_미제_x,거래금액_x,건축년도_x,전용면적_평_x,보증금_x,매매상승률_x,전세상승률_x,전용면적_미제_y,거래금액_y,건축년도_y,전용면적_평_y,보증금_y,매매상승률_y,전세상승률_y
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,NaN,60.0,199.500000,2007.0,18.0,166.666667,0.0,1.010101
1,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,NaN,59.0,141.500000,1997.0,18.0,125.000000,0.0,13.636364
2,경기도 고양덕양구 고양동_윤창_85,2017_1Q,85.0,NaN,1997.0,26.0,NaN,NaN,NaN,85.0,177.425000,1997.0,26.0,123.333333,0.0,0.000000
3,경기도 고양덕양구 고양동_푸른마을 10단지_116,2017_1Q,116.0,NaN,2009.0,35.0,230.000000,NaN,NaN,116.0,336.500000,2009.0,35.0,230.000000,0.0,4.545455
4,경기도 고양덕양구 고양동_푸른마을2단지_135,2017_1Q,135.0,NaN,2004.0,41.0,NaN,NaN,NaN,135.0,310.833333,2004.0,41.0,180.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7054,인천광역시 중구 항동7가_라이프비취2차_129,2017_1Q,129.0,NaN,1981.0,39.0,NaN,NaN,NaN,129.0,195.000000,1981.0,39.0,NaN,0.0,NaN
7055,인천광역시 중구 항동7가_라이프비취2차_52,2017_1Q,52.0,NaN,1981.0,15.0,NaN,NaN,NaN,52.0,81.500000,1981.0,15.0,NaN,0.0,NaN
7056,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,NaN,72.0,120.000000,1981.0,22.0,95.000000,0.0,18.012422
7057,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,NaN,126.0,189.000000,1990.0,38.0,100.000000,0.0,0.000000


In [126]:
# 불필요 피처 제거
df_inner_mae.drop(['거래금액_x','전용면적_미제_y','건축년도_y','전용면적_평_y','보증금_y','매매상승률_y','전세상승률_y'], axis=1, inplace=True)
df_inner_mae

,id,quarter,전용면적_미제_x,건축년도_x,전용면적_평_x,보증금_x,매매상승률_x,전세상승률_x,거래금액_y
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,2007.0,18.0,166.666667,NaN,NaN,199.500000
1,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,1997.0,18.0,125.000000,NaN,NaN,141.500000
2,경기도 고양덕양구 고양동_윤창_85,2017_1Q,85.0,1997.0,26.0,NaN,NaN,NaN,177.425000
3,경기도 고양덕양구 고양동_푸른마을 10단지_116,2017_1Q,116.0,2009.0,35.0,230.000000,NaN,NaN,336.500000
4,경기도 고양덕양구 고양동_푸른마을2단지_135,2017_1Q,135.0,2004.0,41.0,NaN,NaN,NaN,310.833333
...,...,...,...,...,...,...,...,...,...
7054,인천광역시 중구 항동7가_라이프비취2차_129,2017_1Q,129.0,1981.0,39.0,NaN,NaN,NaN,195.000000
7055,인천광역시 중구 항동7가_라이프비취2차_52,2017_1Q,52.0,1981.0,15.0,NaN,NaN,NaN,81.500000
7056,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,1981.0,22.0,95.000000,NaN,NaN,120.000000
7057,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,1990.0,38.0,NaN,NaN,NaN,189.000000


In [127]:
# 피처 이름 수정 및 재배열
df_inner_mae.columns = ['id', 'quarter', '전용면적_미제', '건축년도', '전용면적_평','보증금','매매상승률','전세상승률','거래금액']
df_inner_mae = df_inner_mae[['id', 'quarter', '전용면적_미제', '거래금액', '건축년도', '전용면적_평', '보증금','매매상승률','전세상승률']]
df_inner_mae

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,NaN,NaN
1,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,141.500000,1997.0,18.0,125.000000,NaN,NaN
2,경기도 고양덕양구 고양동_윤창_85,2017_1Q,85.0,177.425000,1997.0,26.0,NaN,NaN,NaN
3,경기도 고양덕양구 고양동_푸른마을 10단지_116,2017_1Q,116.0,336.500000,2009.0,35.0,230.000000,NaN,NaN
4,경기도 고양덕양구 고양동_푸른마을2단지_135,2017_1Q,135.0,310.833333,2004.0,41.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7054,인천광역시 중구 항동7가_라이프비취2차_129,2017_1Q,129.0,195.000000,1981.0,39.0,NaN,NaN,NaN
7055,인천광역시 중구 항동7가_라이프비취2차_52,2017_1Q,52.0,81.500000,1981.0,15.0,NaN,NaN,NaN
7056,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,120.000000,1981.0,22.0,95.000000,NaN,NaN
7057,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,189.000000,1990.0,38.0,NaN,NaN,NaN


In [128]:
# [2017_1Q & 결측치 아닌 데이터] + [df_inner_mae] 
df_notnull= df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.거래금액.isna()==False)]
df_result_mae = pd.concat([df_notnull, df_inner_mae])
df_result_mae.sort_values(['id','quarter'], inplace=True)
df_result_mae.reset_index(drop=True, inplace=True)
df_result_mae

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,NaN,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,NaN
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,NaN,NaN
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,NaN
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,141.500000,1997.0,18.0,125.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,120.000000,1981.0,22.0,95.000000,NaN,NaN
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,189.000000,1990.0,38.0,NaN,NaN,NaN
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,241.666667,1990.0,49.0,NaN,NaN,NaN
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,NaN,NaN


In [129]:
# 매매상승률도 마찬가지 방법으로 merge
df_inner_mae_2 = pd.merge(maeNull_2017_2, df_2014_17_1Q, how ='inner', on =['id','quarter'])
# 불필요한 피처 제거
df_inner_mae_2.drop(['매매상승률_x','거래금액_y','전용면적_미제_y','건축년도_y','전용면적_평_y','보증금_y','전세상승률_y'], axis=1, inplace=True)
df_inner_mae_2

,id,quarter,전용면적_미제_x,거래금액_x,건축년도_x,전용면적_평_x,보증금_x,전세상승률_x,매매상승률_y
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,0.000000
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,-2.316213
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,NaN,1.971831
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,8.208955
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,0.000000
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,0.000000
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,NaN,0.000000
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,NaN,-6.761006


In [130]:
# 피처 이름 수정 및 재배열
df_inner_mae_2.columns = ['id', 'quarter', '전용면적_미제', '거래금액','건축년도', '전용면적_평','보증금','전세상승률','매매상승률']
df_inner_mae_2 = df_inner_mae_2[['id', 'quarter', '전용면적_미제', '거래금액', '건축년도', '전용면적_평', '보증금','매매상승률','전세상승률']]
df_inner_mae_2

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,0.000000,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,-2.316213,NaN
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,1.971831,NaN
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,8.208955,NaN
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,0.000000,NaN
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,0.000000,NaN
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,0.000000,NaN
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,-6.761006,NaN


In [131]:
# [2017_1Q + 전세보증금 결측치가 없는 데이터] + df_inner_jeon
df_notnull= df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.매매상승률.isna()==False)]
df_result_mae_2 = pd.concat([df_notnull, df_inner_mae_2])
df_result_mae_2.sort_values(['id','quarter'], inplace=True)
df_result_mae_2.reset_index(drop=True, inplace=True)
df_result_mae_2

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,0.000000,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,-2.316213,NaN
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,1.971831,NaN
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,8.208955,NaN
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,0.000000,NaN
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,0.000000,NaN
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,0.000000,NaN
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,-6.761006,NaN


In [132]:
# 전월세도 마찬가지 방법으로 merge
df_inner_jeon = pd.merge(jeonNull_2017, df_2014_17_1Q, how ='inner', on =['id','quarter'])

# 불필요한 피처 제거
df_inner_jeon.drop(['보증금_x','거래금액_y','전용면적_미제_y','건축년도_y','전용면적_평_y','매매상승률_y','전세상승률_y'], axis=1, inplace=True)
df_inner_jeon

,id,quarter,전용면적_미제_x,거래금액_x,건축년도_x,전용면적_평_x,매매상승률_x,전세상승률_x,보증금_y
0,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.00,1998.0,18.0,NaN,NaN,140.000000
1,경기도 고양덕양구 고양동_윤창_85,2017_1Q,85.0,NaN,1997.0,26.0,NaN,NaN,123.333333
2,경기도 고양덕양구 고양동_푸른마을 10단지_107,2017_1Q,107.0,301.40,2009.0,32.0,NaN,NaN,200.000000
3,경기도 고양덕양구 고양동_푸른마을1단지_60,2017_1Q,60.0,221.40,2008.0,18.0,NaN,NaN,175.000000
4,경기도 고양덕양구 고양동_푸른마을2단지_135,2017_1Q,135.0,NaN,2004.0,41.0,NaN,NaN,180.000000
...,...,...,...,...,...,...,...,...,...
8331,인천광역시 중구 항동7가_라이프비취2차_129,2017_1Q,129.0,NaN,1981.0,39.0,NaN,NaN,NaN
8332,인천광역시 중구 항동7가_라이프비취2차_52,2017_1Q,52.0,NaN,1981.0,15.0,NaN,NaN,NaN
8333,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,100.000000
8334,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,NaN,100.000000


In [133]:
# 피처 이름 수정 및 재배열
df_inner_jeon.columns = ['id', 'quarter', '전용면적_미제', '거래금액','건축년도', '전용면적_평','매매상승률','전세상승률','보증금']
df_inner_jeon = df_inner_jeon[['id', 'quarter', '전용면적_미제', '거래금액', '건축년도', '전용면적_평', '보증금','매매상승률','전세상승률']]
df_inner_jeon

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.00,1998.0,18.0,140.000000,NaN,NaN
1,경기도 고양덕양구 고양동_윤창_85,2017_1Q,85.0,NaN,1997.0,26.0,123.333333,NaN,NaN
2,경기도 고양덕양구 고양동_푸른마을 10단지_107,2017_1Q,107.0,301.40,2009.0,32.0,200.000000,NaN,NaN
3,경기도 고양덕양구 고양동_푸른마을1단지_60,2017_1Q,60.0,221.40,2008.0,18.0,175.000000,NaN,NaN
4,경기도 고양덕양구 고양동_푸른마을2단지_135,2017_1Q,135.0,NaN,2004.0,41.0,180.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8331,인천광역시 중구 항동7가_라이프비취2차_129,2017_1Q,129.0,NaN,1981.0,39.0,NaN,NaN,NaN
8332,인천광역시 중구 항동7가_라이프비취2차_52,2017_1Q,52.0,NaN,1981.0,15.0,NaN,NaN,NaN
8333,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,100.000000,NaN,NaN
8334,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,100.000000,NaN,NaN


In [134]:
# [2017_1Q + 보증금 결측치가 없는 데이터] + df_inner_jeon
df_notnull= df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.보증금.isna()==False)]
df_result_jeon = pd.concat([df_notnull, df_inner_jeon])
df_result_jeon.sort_values(['id','quarter'], inplace=True)
df_result_jeon.reset_index(drop=True, inplace=True)
df_result_jeon

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,NaN
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,NaN
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,140.000000,NaN,NaN
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,NaN
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,NaN
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,100.000000,NaN,NaN
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,100.000000,NaN,NaN
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,90.000000,NaN,NaN


In [135]:
# 전세상승률도 마찬가지 방법으로 merge
df_inner_jeon_2 = pd.merge(jeonNull_2017_2, df_2014_17_1Q, how ='inner', on =['id','quarter'])

# 불필요한 피처 제거
df_inner_jeon_2.drop(['전세상승률_x','거래금액_y','전용면적_미제_y','건축년도_y','전용면적_평_y','보증금_y','매매상승률_y'], axis=1, inplace=True)
df_inner_jeon_2

,id,quarter,전용면적_미제_x,거래금액_x,건축년도_x,전용면적_평_x,보증금_x,매매상승률_x,전세상승률_y
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,4.824561
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,NaN,0.000000
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,5.882353
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,13.636364
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,18.012422
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,0.000000
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,NaN,0.000000
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,NaN,0.000000


In [136]:
# 피처 이름 수정 및 재배열
df_inner_jeon_2.columns = ['id', 'quarter', '전용면적_미제', '거래금액','건축년도', '전용면적_평','보증금','매매상승률','전세상승률']
df_inner_jeon_2 = df_inner_jeon_2[['id', 'quarter', '전용면적_미제', '거래금액', '건축년도', '전용면적_평', '보증금','매매상승률','전세상승률']]
df_inner_jeon_2

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,4.824561
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,NaN,0.000000
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,5.882353
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,13.636364
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,18.012422
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,0.000000
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,NaN,0.000000
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,NaN,0.000000


In [137]:
# [2017_1Q + 보증금 결측치가 없는 데이터] + df_inner_jeon
df_notnull= df_merge_2017[(df_merge_2017.quarter == '2017_1Q') & (df_merge_2017.전세상승률.isna()==False)]
df_result_jeon_2 = pd.concat([df_notnull, df_inner_jeon_2])
df_result_jeon_2.sort_values(['id','quarter'], inplace=True)
df_result_jeon_2.reset_index(drop=True, inplace=True)
df_result_jeon_2

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,NaN,2007.0,18.0,166.666667,NaN,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,NaN,4.824561
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,NaN,NaN,0.000000
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,NaN,5.882353
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,NaN,1997.0,18.0,125.000000,NaN,13.636364
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,NaN,1981.0,22.0,95.000000,NaN,18.012422
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,NaN,1990.0,38.0,NaN,NaN,0.000000
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,NaN,1990.0,49.0,NaN,NaN,0.000000
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,NaN,NaN,0.000000


In [138]:
# 최종 병합 데이터 = df_result_mae에서 보증금만 제외 + df_result_jeon의 보증금
df_result = pd.concat([df_result_mae.iloc[:, :-3], df_result_jeon.iloc[:,-3],df_result_mae_2.iloc[:,-2],df_result_jeon_2.iloc[:,-1]], axis=1)
df_result

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2017_1Q,85.0,242.500000,2007.0,26.0,199.166667,-2.316213,4.824561
2,경기도 고양덕양구 고양동_삼성_60,2017_1Q,60.0,181.000000,1998.0,18.0,140.000000,1.971831,0.000000
3,경기도 고양덕양구 고양동_삼성_85,2017_1Q,85.0,217.500000,1998.0,26.0,180.000000,8.208955,5.882353
4,경기도 고양덕양구 고양동_윤창_59,2017_1Q,59.0,141.500000,1997.0,18.0,125.000000,0.000000,13.636364
...,...,...,...,...,...,...,...,...,...
21511,인천광역시 중구 항동7가_라이프비취2차_72,2017_1Q,72.0,120.000000,1981.0,22.0,95.000000,0.000000,18.012422
21512,인천광역시 중구 항동7가_라이프비취3차_126,2017_1Q,126.0,189.000000,1990.0,38.0,100.000000,0.000000,0.000000
21513,인천광역시 중구 항동7가_라이프비취3차_164,2017_1Q,164.0,241.666667,1990.0,49.0,100.000000,0.000000,0.000000
21514,인천광역시 중구 항동7가_라이프비취3차_85,2017_1Q,85.0,148.250000,1990.0,26.0,90.000000,-6.761006,0.000000


### `df_result`와 `df_merge_2017` 2017_2Q ~ 2021_2Q 데이터를 병합

In [139]:
# 2017년도 1분기 갈아끼우기
# 2017_1Q ~ 2021_2Q 리스트
over_2017_1Q = ['2017_2Q', '2017_3Q', '2017_4Q',
                '2018_1Q', '2018_2Q', '2018_3Q', '2018_4Q',
                '2019_1Q', '2019_2Q', '2019_3Q', '2019_4Q',
                '2020_1Q', '2020_2Q', '2020_3Q', '2020_4Q',
                '2021_1Q', '2021_2Q']
# 2017~2021 데이터에서 분기가 2017_2Q~2021_2Q인 데이터
df_merge_2017_over = df_merge_2017[df_merge_2017.quarter.isin(over_2017_1Q)]

# 2014~2021 데이터의 2017년도 1분기와 2017~2021 데이터에서 분기가 2017 2분기~2021 2분기인 데이터를 merge
df_merge_new = pd.concat([df_result, df_merge_2017_over])

# 정렬
df_merge_new.sort_values(by=['id','quarter'],inplace=True)
df_merge_new.reset_index(drop=True, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,NaN,0.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667


In [140]:
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        6714
건축년도           0
전용면적_평         0
보증금        27705
매매상승률      14513
전세상승률      35857
dtype: int64

#### 2017_1Q 결측치 개수 똑같은지 확인

**거래금액: 839개**

**보증금: 1701개**

In [141]:
# 거기서 거래금액 0원인거
display(df_merge_new[(df_merge_new.quarter == '2017_1Q') & (df_merge_new.거래금액.isna())].shape)

# 거기서 보증금 0원인거
display(df_merge_new[(df_merge_new.quarter == '2017_1Q') & (df_merge_new.보증금.isna())].shape)

(839, 9)

(1701, 9)

#  4. 다시 이전 분기로 채우기

In [142]:
# 이전분기 채우기
df_merge_new['거래금액'] = df_merge_new.groupby('id')['거래금액'].ffill()
df_merge_new[['보증금']] = df_merge_new.groupby('id')[['보증금']].ffill()
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,NaN,0.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667


In [143]:
# 결측치 확인
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        2174
건축년도           0
전용면적_평         0
보증금        12339
매매상승률      14513
전세상승률      35857
dtype: int64

# 5. 건축년도 2017년 이상인 id에 대해 초기 분양권/입주권 데이터로 채우기
- `insert_bunyang`에 분양권/입주권 데이터가 적용된 데이터를 저장했었다.
- 건축년도가 2017년 이상이면서 `insert_bunyang`의 id를 갖는 데이터에 대해 bfill

In [144]:
# 분양권/입주권 데이터가 적용된 데이터
insert_bunyang

,id,quarter,본번,부번,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,분양권/입주권
1080,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_1Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,383.206250
1081,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393.983056
1082,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.584103
1083,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2017_4Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413.975100
1084,경기도 고양덕양구 도내동_고양원흥동일스위트_85,2018_1Q,NaN,NaN,NaN,NaN,NaN,NaN,305.516129,414.315833
...,...,...,...,...,...,...,...,...,...,...
767075,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_2Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329.000000
767076,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_3Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329.915000
767077,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2018_4Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325.500000
767078,인천광역시 중구 중산동_영종하늘도시 KCC스위첸_85,2019_1Q,NaN,NaN,NaN,NaN,NaN,NaN,20.000000,323.185000


In [145]:
# 건축년도가 2017년 이상이면서 insert_bunyang의 id를 갖는 데이터에 대해 bfill
idx_condition = df_merge_new.loc[(df_merge_new.id.isin(insert_bunyang.id.unique())) & (df_merge_new.건축년도 >= 2017)].index
df_merge_new.loc[idx_condition, '거래금액'] = df_merge_new.loc[idx_condition].거래금액.bfill()
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,NaN,0.000000
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667


# ※ 매매상승률, 전세상승률 갱신

In [146]:
# 이전거래금액 컬럼 생성.
df_merge_new['이전거래금액'] = df_merge_new.거래금액.shift()
# # 2017년도 첫 분기는 0이 되게 설정
df_merge_new.loc[(df_merge_new.quarter == '2017_1Q') , '이전거래금액'] = None

# 이전보증금 컬럼 생성
df_merge_new['이전보증금'] = df_merge_new.보증금.shift()
# 2017년도 첫 분기는 0이 되게 설정
df_merge_new.loc[(df_merge_new.quarter == '2017_1Q'), '이전보증금'] = None


# 매매상승률, 전세상승률 컬럼 생성
df_merge_new.loc[df_merge_new.매매상승률.isna(), '매매상승률'] = (df_merge_new.거래금액 - df_merge_new.이전거래금액) * 100 / df_merge_new.이전거래금액 
df_merge_new.loc[df_merge_new.전세상승률.isna(), '전세상승률'] = (df_merge_new.보증금 - df_merge_new.이전보증금) * 100 / df_merge_new.이전보증금

# 2017년도 첫 분기는 0이 되게 설정
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.매매상승률.isna()), '매매상승률'] = 0
# df_merge.loc[(df_merge.quarter == '2017_1Q')&(df_merge.전세상승률.isna()), '전세상승률'] = 0

# 이전거래금액, 이전보증금 삭제
df_merge_new.drop(['이전거래금액','이전보증금'], axis=1, inplace=True)
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        1173
건축년도           0
전용면적_평         0
보증금        12339
매매상승률       2752
전세상승률      14037
dtype: int64

# 6. 컬럼추가
- 전세가율
- 평당매매가
- 매매빈도
- 위도, 경도
- 서울구분
- 6억 초과
- 매매상승률, 전세상승률
- 규제
- 평당거주비용

In [150]:
# 매매데이터 전세데이터 불러오기
df_mae = pd.read_csv('data/MasterMotherTable/df_mae_1721_11월04일17시34분.csv')
df_mae.rename(columns={'quater':'quarter'}, inplace=True)

df_jeonwol = pd.read_csv('data/MasterMotherTable/df_jeonwol_1721_11월04일17시34분.csv')
df_jeonwol.rename(columns={'quater':'quarter'}, inplace=True)

# -------------------------------------------------------------------------------------------
# 위도 경도 데이터 불러오기
latlong = pd.read_csv('data/df_위도경도_11.8.csv')
latlong
# -------------------------------------------------------------------------------------------
# 계약년월과 계약일로 시계열변수 new_Date
def monthToQuarter(df):
    # Series 자료형을 Object로 변경
    df['계약년월'] = df['계약년월'].astype('O')
 
    q = []                                         # quater 데이터를 담을 리스트
    for i in range(len(df.index.values)):
        year = str(df['계약년월'].iloc[i])[0:4]    # 년
        month = str(df['계약년월'].iloc[i])[4:]    # 월
        
        # month를 숫자로 바꿔서 부등호조건 활용해서 쿼터 나누고 리스트에 담기.
        month = int(month)
        if month < 4:
            q.append(year+'_1Q')
        elif month < 7:
            q.append(year+'_2Q')
        elif month < 10:
            q.append(year+'_3Q')
        elif month < 13:
            q.append(year+'_4Q')
               
    # 파생변수 quater 생성
    df['quarter'] = q
    
    return df

# 주택담보대출금리 데이터 가져오기
df_dambo = pd.read_excel('data/주택담보대출금리.xlsx')
df_dambo.drop('지 역', axis=1, inplace=True)

# 데이터 전처리
df_dambo = df_dambo.T.reset_index(drop=False)
df_dambo.columns = ['계약년월', '주택담보대출금리']
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('년','')
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('월','')
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace(' ','')

df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('. ','.')
df_dambo['계약년월'] = df_dambo['계약년월'].apply(lambda x: int(x))
df_dambo = monthToQuarter(df_dambo)

# 분기별 평균 금리구하기
df_dambo_mean = df_dambo.groupby('quarter', as_index=False).mean()
df_dambo_mean
# -------------------------------------------------------------------------------------------

C:\Users\medici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-150-cb30066afa7c>:50: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('. ','.')


,quarter,주택담보대출금리
0,2016_1Q,3.020000
1,2016_2Q,2.863333
2,2016_3Q,2.720000
3,2016_4Q,3.020000
4,2017_1Q,3.186667
5,2017_2Q,3.230000
6,2017_3Q,3.266667
7,2017_4Q,3.380000
8,2018_1Q,3.460000
9,2018_2Q,3.473333


## 6-1. `전세가율` 컬럼 추가 
$$ 전세가율 = \frac{전세보증금}{거래금액}$$

- `0/0 = NaN` 인 경우가 존재. 이 경우 0으로 대체

In [151]:
# 전월세가 = 보증금 / 거래금액
df_merge_new['전세가율'] = df_merge_new.보증금 * 100 / df_merge_new.거래금액
df_merge_new['전세가율'].fillna(0, inplace = True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713
...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789


## 6-2. `평당매매가` 컬럼 추가
$$ 평당매매가 = \frac{거래금액}{평수} $$

In [152]:
# 평당매매가 = 거래금액 / 평수
df_merge_new['평당매매가'] = df_merge_new.거래금액 / df_merge_new.전용면적_평
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111
...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500


## 6-3. `매매빈도` 컬럼 추가

In [153]:
# Grouping
df_mae_g = df_mae.groupby(['id','quarter'],as_index=False)['거래금액'].count()
df_mae_g.rename(columns={'거래금액':'매매빈도'}, inplace=True)

# df_merge_new와 df_mae_g를 left join하여 매매빈도 추가
df_merge_new = pd.merge(df_merge_new, df_mae_g, how='left', on = ['id','quarter'])

# 결측치는 0으로 대체
df_merge_new['매매빈도'].fillna(0, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0


In [154]:
df_merge_new.id.unique().shape

(21516,)

### 6-5 전세빈도 추가

In [155]:
# Grouping
df_jeonwol_g = df_jeonwol.groupby(['id','quarter'],as_index=False)['보증금'].count()
df_jeonwol_g.rename(columns={'보증금':'전세빈도'}, inplace=True)

# df_merge_new와 df_mae_g를 left join하여 매매빈도 추가
df_merge_new = pd.merge(df_merge_new, df_jeonwol_g, how='left', on = ['id','quarter'])

# 결측치는 0으로 대체
df_merge_new['전세빈도'].fillna(0, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0


## 6-4. 위도, 경도 컬럼 추가

In [156]:
# 마스터테이블과 위도경도 테이블 merge
df_merge_new = pd.merge(df_merge_new, latlong, how='left', on='id') 
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0,37.700477,126.899362
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0,37.700477,126.899362
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0,37.700477,126.899362
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0,37.700477,126.899362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0,37.448902,126.620839
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0,37.448902,126.620839
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0,37.448902,126.620839
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0,37.448902,126.620839


In [157]:
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        1173
건축년도           0
전용면적_평         0
보증금        12339
매매상승률       2752
전세상승률      14037
전세가율           0
평당매매가       1173
매매빈도           0
전세빈도           0
위도          7002
경도          7002
dtype: int64

## 6-5.서울구분 컬럼 추가
- 서울이면 1, 경기&인천이면 0

In [158]:
# df_merge_new와 df_mae를 left join하여 지역 컬럼 추가
df_merge_new = pd.merge(df_merge_new, df_mae[['id','지역']], how='left', on ='id')

# 서울 구분 컬럼 추가
df_merge_new['서울구분']=0
df_merge_new.loc[df_merge_new.지역 =='서울', '서울구분'] = 1
df_merge_new.drop('지역', axis=1, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.00000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.00000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.00000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.00000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.00000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19976413,인천광역시 중구 항동7가_연안아파트_54,2021_2Q,54.0,245.857143,1983.0,16.0,20.000000,5.80872,-1.639344,8.134805,15.366071,14.0,3.0,37.448902,126.620839,0
19976414,인천광역시 중구 항동7가_연안아파트_54,2021_2Q,54.0,245.857143,1983.0,16.0,20.000000,5.80872,-1.639344,8.134805,15.366071,14.0,3.0,37.448902,126.620839,0
19976415,인천광역시 중구 항동7가_연안아파트_54,2021_2Q,54.0,245.857143,1983.0,16.0,20.000000,5.80872,-1.639344,8.134805,15.366071,14.0,3.0,37.448902,126.620839,0
19976416,인천광역시 중구 항동7가_연안아파트_54,2021_2Q,54.0,245.857143,1983.0,16.0,20.000000,5.80872,-1.639344,8.134805,15.366071,14.0,3.0,37.448902,126.620839,0


## 6-6. 6억 초과 여부 컬럼 추가
- 초과시 1, 그렇지 않으면 0

In [159]:
# 6억 컬럼 추가
df_merge_new['6억초과'] = 0
df_merge_new.loc[df_merge_new.거래금액 > 600, '6억초과'] = 1
df_merge_new.drop_duplicates(inplace=True)
df_merge_new.sort_values(['id','quarter'], inplace=True)
df_merge_new.reset_index(drop=True, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분,6억초과
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0,0
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0,37.700477,126.899362,0,0
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0,37.700477,126.899362,0,0
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0,37.700477,126.899362,0,0
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0,37.700477,126.899362,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0,37.448902,126.620839,0,0
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0,37.448902,126.620839,0,0
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0,37.448902,126.620839,0,0
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0,37.448902,126.620839,0,0


## 6-8. 규제 컬럼 생성

In [160]:
#투기과열지구
def find_region(x):
    if "서울" in x['시군구']:
        if (('강남구' in x['시군구']) or ('서초' in x['시군구']) or ('송파' in x['시군구'])or ('강동' in x['시군구'])or ('용산' in x['시군구'])or('성동' in x['시군구'])or ('노원' in x['시군구'])or ('마포' in x['시군구'])or ('양천' in x['시군구'])or ('영등포' in x['시군구'])or ('강서' in x['시군구'])) and (x['quarter'] >='2017_3Q'):
            return '투기지구'
        elif (('종로' in x['시군구'])or ('중' in x['시군구'])or ('동대문' in x['시군구'])or ('동작' in x['시군구'])) and (x['quarter'] >='2018_3Q'):
            return '투기지구'
        else:
            return '투기과열지구'
        
    if "경기도" in x['시군구']:
        #ok
        if "과천" in x['시군구']:
            if x['quarter'] >='2017_3Q':
                return '투기과열지구'
#             elif x['quarter'] >='2017_3Q':
#                 return '조정대상지역'
            else:
                return '해당 없음'
        #ok    
        elif "군포" in x['시군구']:
            if x['quarter'] >='2020_3Q':
                return '투기과열지구'
#             elif x['quarter'] >='2020_3Q':
#                 return '조정대상지역'
            else:
                return '해당 없음'
        #ok
        elif "성남" in x['시군구']:
            if ('분당구' in x['시군구']):
                if (x['quarter'] >='2017_4Q'):
                    return '투기과열지구'
                else:
                    return '해당 없음'
            elif ('수정구'in x['시군구']):
                if (x['quarter'] >='2020_3Q'):
                    return '투기과열지구'
                else:
                    return '해당 없음'
            #201611이후부터 성남 전체 조정대상지역
            else:
                return '조정대상지역'

        #ok
        elif "광명" in x['시군구']:
            if x['quarter'] >='2018_3Q':
                return '투기과열지구'
            elif x['quarter'] >='2017_4Q':
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok
        elif "하남" in x['시군구']:
            if x['quarter'] >='2018_3Q':
                return '투기과열지구'
            elif x['quarter'] >='2017_4Q':
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok    
        elif "구리" in x['시군구']:
            if x['quarter'] >='2020_3Q':
                return '투기과열지구'
            elif x['quarter'] >='2018_3Q':
                return '조정대상지역'
            else:
                return '해당 없음' 
        #ok
        elif "안양" in x['시군구']:
            if ('안양'in x['시군구']) and (x['quarter'] >='2020_3Q'):
                return '투기과열지구'
            elif ('만안구'in x['시군구']) and (x['quarter'] >='2020_1Q'):
                return '조정대상지역'
            elif ('동안구'in x['시군구']) and (x['quarter'] >='2018_3Q'):
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok
        #광교택지개발지구(수원시 영통구 이의동, 원천동, 하동, 매탄동, 팔달구 우만동, 장안구 연무동 -> 이 중 일원이지만  하나하나 구할 순 없기에,,
        elif "수원" in x['시군구']:
            if (x['quarter'] >='2020_3Q'):
                return '투기과열지구'
            elif (('영통구'in x['시군구']) or ('권선구'in x['시군구']) or ('장안구'in x['시군구'])) and (x['quarter'] >='2020_1Q'):
                return '조정대상지역'
            
            elif ('팔달구'in x['시군구']) and (x['quarter'] >='2019_1Q'):
                return '조정대상지역'
            #광교택지개발지구
            elif (('이의동'in x['시군구']) or ('원천동'in x['시군구']) or ('하동'in x['시군구'])or ('매탄동'in x['시군구'])or ('우만동'in x['시군구'])or ('연무동'in x['시군구'])) and (x['quarter'] >='2018_3Q'):
                return '조정대상지역'
            else:
                return '해당 없음'
         #ok   
        elif "의왕" in x['시군구']:
            if x['quarter'] >='2020_3Q':
                return '투기과열지구'
            elif x['quarter'] >='2020_1Q':
                return '조정대상지역'
            else:
                return '해당 없음' 
        #ok    
        elif "용인" in x['시군구']:
            if ('수지구'in x['시군구']) or ('기흥구'in x['시군구']):
                if(x['quarter'] >='2020_3Q'):
                    return '투기과열지구'
                #광교택지개발지구
                elif(('상현동'in x['시군구']) or ('영덕동'in x['시군구'])) and (x['quarter'] >='2019_1Q'):
                    return '조정대상지역'
                else:
                    return '해당 없음'
                
            elif ('처인구' in x['시군구']) and (x['quarter'] >='2020_3Q'):
                #포곡읍, 모현읍, 백암면,양지면 및 원삼면 가재월리ㆍ사암리ㆍ미평리ㆍ좌항리ㆍ맹리ㆍ두창리 제외
                if (("포곡읍" not in x['시군구']) or ("모현읍" not in x['시군구'])or ("백암면" not in x['시군구']) or ("양지면" not in x['시군구']) or ("원삼면" not in x['시군구']) or ("가재월리" not in x['시군구'])or ("사암리" not in x['시군구'])or ("미평리" not in x['시군구'])or ("좌항리" not in x['시군구'])or ("맹리" not in x['시군구'])or ("두창리" not in x['시군구']) and (x['quarter'] >='2020_3Q')):
                    return '조정대상지역'
                else:
                    return '해당 없음'
            else:
                return '해당 없음'
        #ok     
        elif "화성" in x['시군구']:
            if (('반송동' in x['시군구']) or ('석우동'in x['시군구'])or ('금곡리'in x['시군구'])or ('목리'in x['시군구'])or ('방교리'in x['시군구'])or ('산척리'in x['시군구'])or ('송리'in x['시군구'])or ('신리'in x['시군구'])or ('영천리'in x['시군구'])or ('오산리'in x['시군구'])or ('장지리'in x['시군구'])or ('중리'in x['시군구'])or ('청계리'in x['시군구'])):
                if x['quarter'] >='2020_3Q':
                    return '투기과열지구'
                elif x['quarter'] >='2017_4Q':
                    return '조정대상지역'
                else:
                    return '해당 없음'
            elif(x['quarter'] >='2020_3Q'):
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok 
        elif ("부천" in x['시군구'] or "시흥" in x['시군구'] or "오산" in x['시군구'] or "평택" in x['시군구'] or "광주" in x['시군구'] or "의정부" in x['시군구']):
            if x['quarter'] >='2020_3Q':
                return '조정대상지역'
            else:
                return '해당 없음' 
            
        #ok     
        #안성시 조정대상지역 지정, 해제, 재지정 반영    
        elif ("안성" in x['시군구']):    
            if ((x['quarter'] >='2020_3Q') and (x['quarter'] <='2020_4Q')):
                return '조정대상지역'
            else:
                return '해당 없음'
            
            
            
        #남양주 조정대상지역 지정, 해제, 재지정 반영    
        elif ("남양주" in x['시군구']):    
            if (x['quarter'] >='2020_3Q') or ((x['quarter'] >='2017_4Q') and (x['quarter'] <='2019_4Q')):
                return '조정대상지역'
            else:
                return '해당 없음'
            
        #ok    
        #양주시 조정대상지역 지정, 해제, 재지정 반영    
        elif ("양주" in x['시군구']):    
            if ((x['quarter'] >='2020_3Q') and (x['quarter'] <='2020_4Q')):
                return '조정대상지역'
            else:
                return '해당 없음'
         #ok      
        #고양 조정대상지역 지정, 해제, 재지정 반영    
        elif ("고양" in x['시군구']):
            if x['quarter'] >='2020_3Q' :
                return '조정대상지역'
            elif "삼송" not in x['시군구'] or "원흥" not in x['시군구'] or "지축" not in x['시군구'] or "향동" not in x['시군구'] or "덕은" not in x['시군구']:
                if ((x['quarter'] >='2017_4Q') and (x['quarter'] <='2019_4Q')) or (x['quarter'] >='2020_3Q'):
                    return '조정대상지역'
                else:
                    return '해당 없음'
            else:
                return '해당 없음'


        #ok     
        elif ("김포" in x['시군구']):
            if x['quarter'] >='2020_4Q':
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok
        elif ("파주" in x['시군구']):
            if x['quarter'] >='2021_1Q':
                return '조정대상지역'
            else:
                return '해당 없음'
        #ok 
        elif ("안산" in x['시군구']):
            if ("단원구" in x['시군구'])and (x['quarter'] >='2020_3Q'):
                return '투기과열지구'
            elif x['quarter'] >='2020_3Q':
                return '조정대상지역'
            else:
                return '해당 없음'

    if "인천" in x['시군구']:
        if ((("연수구" in x['시군구'])or("남동구" in x['시군구'])or("서구" in x['시군구'])))and (x['quarter'] >='2020_3Q'):
            return '투기과열지구'
        elif((("인천광역시 중구" in x['시군구'])or("인천광역시 동구" in x['시군구'])or("인천광역시 미추홀구" in x['시군구'])or("인천광역시 부평구" in x['시군구'])or("인천광역시 계양구" in x['시군구']))) and (x['quarter'] >='2020_3Q'):
            if ('강화군' not in x['시군구']) and ('옹진군' not in x['시군구']):
                return '조정대상지역'
        else:
            return '해당 없음'


In [161]:
# df_merge_new에 df_mae를 left join하여 시군구 컬럼 생성
df_merge_new = pd.merge(df_merge_new, df_mae[['id', '시군구']], how='left', on=['id'])
df_merge_new.drop_duplicates(inplace=True)
df_merge_new.sort_values(['id','quarter'], inplace=True)
df_merge_new.reset_index(drop=True, inplace=True)
df_merge_new.rename(columns={'시군구_x':'시군구'},inplace=True)

# 규제컬럼 생성
df_merge_new["규제"] = df_merge_new.apply(lambda x:find_region(x),axis=1)
df_merge_new.drop('시군구', axis=1, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,전용면적_평,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분,6억초과,규제
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,18.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0,0,해당 없음
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,18.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0,37.700477,126.899362,0,0,해당 없음
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,18.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0,37.700477,126.899362,0,0,해당 없음
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,18.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0,37.700477,126.899362,0,0,조정대상지역
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,18.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0,37.700477,126.899362,0,0,조정대상지역
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,16.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0,37.448902,126.620839,0,0,해당 없음
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,16.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0,37.448902,126.620839,0,0,조정대상지역
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,16.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0,37.448902,126.620839,0,0,조정대상지역
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,16.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0,37.448902,126.620839,0,0,조정대상지역


In [162]:
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        1173
건축년도           0
전용면적_평         0
보증금        12339
매매상승률       2752
전세상승률      14037
전세가율           0
평당매매가       1173
매매빈도           0
전세빈도           0
위도          7002
경도          7002
서울구분           0
6억초과           0
규제             0
dtype: int64

### 6-9. 평당거주비용 컬럼 생성

In [163]:
# 계약년월과 계약일로 시계열변수 new_Date
def monthToQuarter(df):
    # Series 자료형을 Object로 변경
    df['계약년월'] = df['계약년월'].astype('O')
 
    q = []                                         # quater 데이터를 담을 리스트
    for i in range(len(df.index.values)):
        year = str(df['계약년월'].iloc[i])[0:4]    # 년
        month = str(df['계약년월'].iloc[i])[4:]    # 월
        
        # month를 숫자로 바꿔서 부등호조건 활용해서 쿼터 나누고 리스트에 담기.
        month = int(month)
        if month < 4:
            q.append(year+'_1Q')
        elif month < 7:
            q.append(year+'_2Q')
        elif month < 10:
            q.append(year+'_3Q')
        elif month < 13:
            q.append(year+'_4Q')
               
    # 파생변수 quater 생성
    df['quarter'] = q
    
    return df

# 주택담보대출금리 데이터 가져오기
df_dambo = pd.read_excel('data/주택담보대출금리.xlsx')
df_dambo.drop('지 역', axis=1, inplace=True)

# 데이터 전처리
df_dambo = df_dambo.T.reset_index(drop=False)
df_dambo.columns = ['계약년월', '주택담보대출금리']
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('년','')
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('월','')
df_dambo['계약년월'] = df_dambo['계약년월'].str.replace(' ','')

df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('. ','.')
df_dambo['계약년월'] = df_dambo['계약년월'].apply(lambda x: int(x))
df_dambo = monthToQuarter(df_dambo)

# 분기별 평균 금리구하기
df_dambo_mean = df_dambo.groupby('quarter', as_index=False).mean()
df_dambo_mean

<ipython-input-163-6eb8f1ce4ab4>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dambo['계약년월'] = df_dambo['계약년월'].str.replace('. ','.')


,quarter,주택담보대출금리
0,2016_1Q,3.020000
1,2016_2Q,2.863333
2,2016_3Q,2.720000
3,2016_4Q,3.020000
4,2017_1Q,3.186667
5,2017_2Q,3.230000
6,2017_3Q,3.266667
7,2017_4Q,3.380000
8,2018_1Q,3.460000
9,2018_2Q,3.473333


In [164]:
# df_merge_new에 df_dambo_mean를 left join하여 주택담보대출금리 컬럼 생성
df_merge_new = pd.merge(df_merge_new, df_dambo_mean, how='left', on=['quarter'])
df_merge_new.drop_duplicates(inplace=True)
df_merge_new.sort_values(['id','quarter'], inplace=True)
df_merge_new.reset_index(drop=True, inplace=True)

# 평당거주비용 컬럼생성
df_merge_new['평당거주비용'] = df_merge_new.거래금액 * df_merge_new.주택담보대출금리 / (df_merge_new.전용면적_평*100)
# 주택담보대출금리, 전용면적_평 컬럼 삭제
df_merge_new.drop(['주택담보대출금리', '전용면적_평'], axis=1, inplace=True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분,6억초과,규제,평당거주비용
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0,0,해당 없음,0.353189
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0,37.700477,126.899362,0,0,해당 없음,0.364272
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0,37.700477,126.899362,0,0,해당 없음,0.364173
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0,37.700477,126.899362,0,0,조정대상지역,0.392456
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0,37.700477,126.899362,0,0,조정대상지역,0.401744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0,37.448902,126.620839,0,0,해당 없음,0.362545
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0,37.448902,126.620839,0,0,조정대상지역,0.318500
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0,37.448902,126.620839,0,0,조정대상지역,0.324949
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0,37.448902,126.620839,0,0,조정대상지역,0.388235


In [165]:
# 결측치확인
df_merge_new.isna().sum()

id             0
quarter        0
전용면적_미제        0
거래금액        1173
건축년도           0
보증금        12339
매매상승률       2752
전세상승률      14037
전세가율           0
평당매매가       1173
매매빈도           0
전세빈도           0
위도          7002
경도          7002
서울구분           0
6억초과           0
규제             0
평당거주비용      1173
dtype: int64

# 7. 나머지 거래금액 결측치가 있는 id에 대해 데이터 삭제

In [166]:
# 결측치가 존재하는 id 리스트
remove_id = df_merge_new[df_merge_new.거래금액.isna()].id.unique()
# id 리스트에 해당되는 데이터 인덱스
remove_index =df_merge_new[df_merge_new.id.isin(remove_id)].index

# 결측치가 존재하는 데이터 따로 저장해놓기
df_dropped = df_merge_new.loc[remove_index]
df_dropped.to_csv('df_dropped.csv', index=False, encoding='utf-8-sig')

df_merge_new.drop(remove_index, inplace = True)
df_merge_new

,id,quarter,전용면적_미제,거래금액,건축년도,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분,6억초과,규제,평당거주비용
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_1Q,60.0,199.500000,2007.0,166.666667,0.000000,1.010101,83.542189,11.083333,0.0,6.0,37.700477,126.899362,0,0,해당 없음,0.353189
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_2Q,60.0,203.000000,2007.0,166.666667,1.754386,0.000000,82.101806,11.277778,2.0,0.0,37.700477,126.899362,0,0,해당 없음,0.364272
2,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_3Q,60.0,200.666667,2007.0,175.000000,-1.149425,5.000000,87.209302,11.148148,3.0,3.0,37.700477,126.899362,0,0,해당 없음,0.364173
3,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2017_4Q,60.0,209.000000,2007.0,170.000000,4.152824,-2.857143,81.339713,11.611111,1.0,2.0,37.700477,126.899362,0,0,조정대상지역,0.392456
4,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,2018_1Q,60.0,209.000000,2007.0,170.000000,0.000000,0.000000,81.339713,11.611111,4.0,3.0,37.700477,126.899362,0,0,조정대상지역,0.401744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387283,인천광역시 중구 항동7가_연안아파트_54,2020_2Q,54.0,229.277778,1983.0,21.666667,8.392082,-5.797101,9.449964,14.329861,18.0,4.0,37.448902,126.620839,0,0,해당 없음,0.362545
387284,인천광역시 중구 항동7가_연안아파트_54,2020_3Q,54.0,210.000000,1983.0,18.285714,-8.408045,-15.604396,8.707483,13.125000,3.0,9.0,37.448902,126.620839,0,0,조정대상지역,0.318500
387285,인천광역시 중구 항동7가_연안아파트_54,2020_4Q,54.0,204.692308,1983.0,20.000000,-2.527473,9.375000,9.770763,12.793269,13.0,2.0,37.448902,126.620839,0,0,조정대상지역,0.324949
387286,인천광역시 중구 항동7가_연안아파트_54,2021_1Q,54.0,232.360000,1983.0,20.333333,13.516723,1.666667,8.750789,14.522500,25.0,5.0,37.448902,126.620839,0,0,조정대상지역,0.388235


# 8. 나머지 모두 0 대체

In [170]:
df_merge_new.fillna(0,inplace=True)
df_merge_new.isna().sum()

id         0
quarter    0
전용면적_미제    0
거래금액       0
건축년도       0
보증금        0
매매상승률      0
전세상승률      0
전세가율       0
평당매매가      0
매매빈도       0
전세빈도       0
위도         0
경도         0
서울구분       0
6억초과       0
규제         0
평당거주비용     0
dtype: int64

# 데이터 저장

In [172]:
# 오늘 날짜
today = datetime.today().strftime("%Y%m%d%H%M%S")

# '데이터_날짜' 형태로 csv 저장
df_merge_new.to_csv('data/MasterTable/MTF_비인도데프_'+ today +'.csv', index=False, encoding='utf-8-sig')